In [1]:
print(1)

1


In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat

from config import CONFIG


giga = GigaChat(
    # auth_url = "https://sm-auth-sd.prom-88-89-apps.ocp-geo.ocp.sigma.sbrf.ru/api/v2/oauth",
    scope = 'GIGACHAT_API_CORP',
    # Для авторизации запросов используйте ключ, полученный в проекте GigaChat API
    credentials=CONFIG["token"]["gigachat"],
    verify_ssl_certs=False,
    model = "GigaChat-2-Max"
)
messages = "privet"
res = giga.invoke(messages)
print(res)


content='Привет!' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 12, 'completion_tokens': 4, 'total_tokens': 16, 'precached_prompt_tokens': 2}, 'model_name': 'GigaChat-2-Max:2.0.28.2', 'x_headers': {'x-request-id': '5d981e7c-bbbe-4938-8aee-183d2fec4924', 'x-session-id': '9a0683d1-f042-4d28-82b1-eb1a242009f9', 'x-client-id': None}, 'finish_reason': 'stop'} id='5d981e7c-bbbe-4938-8aee-183d2fec4924' usage_metadata={'output_tokens': 4, 'input_tokens': 12, 'total_tokens': 16, 'input_token_details': {'cache_read': 2}}


In [3]:
import streamlit as st
import whisper
import os
import torch
import traceback
import tempfile
import numpy as np
import torchaudio
import librosa
import soundfile as sf
from st_audiorec import st_audiorec
from langchain_gigachat.chat_models import GigaChat
from transformers import Wav2Vec2FeatureExtractor, UniSpeechSatForAudioFrameClassification
from sklearn.cluster import AgglomerativeClustering
from config import CONFIG
from tqdm import tqdm

giga = GigaChat(
    scope='GIGACHAT_API_CORP',
    credentials=CONFIG["token"]["gigachat"],
    verify_ssl_certs=False,
    model="GigaChat-2-Max"
)

torch.classes.__path__ = []

def generate_detailed_analysis_prompt(transcription_text):
    prompt = f"""  
Обработай транскрипт рабочего совещания. Сначала попробуй определить участников по контексту (диаризация по наитию), группируя реплики по условным ролям или именам (например: "Иван", "Участник 1", "Менеджер проекта"). Затем выдели: 1) Основные темы/проблемы (кратко суть), 2) Поручения (что, кому/роли, сроки если есть), 3) Ключевые решения, 4) Важные детали (цифры, риски, контакты). Не используй markdown. Если списки - только текстовые обозначения типа "а)" или "-". Группируй информацию по смыслу. Структура ответа: сначала условные говорящие, затем остальные категории. Если диаризация невозможна - пропусти её и выдели только суть.

Транскрипт:
{transcription_text}  

""" 
    return prompt

@st.cache_resource
def load_models():
    whisper_model = whisper.load_model("large-v3")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/unispeech-sat-base-plus")
    embedding_model = UniSpeechSatForAudioFrameClassification.from_pretrained("microsoft/unispeech-sat-base-plus")
    
    return whisper_model, feature_extractor, embedding_model

def analyze_transcription(text):
    try:
        prompt = generate_detailed_analysis_prompt(text)
        analysis_result = giga.invoke(prompt)
        return analysis_result.content
    except Exception as e:
        st.error(f"Ошибка при анализе текста: {str(e)}")
        return None

def save_audio(audio_bytes, filename):
    try:
        if not audio_bytes or len(audio_bytes) < 1024:
            st.warning("Аудиоданные слишком короткие или отсутствуют")
            return None
            
        with open(filename, "wb") as f:
            f.write(audio_bytes)
        
        if os.path.getsize(filename) < 1024:
            st.warning("Созданный аудиофайл слишком мал")
            return None
            
        return filename
    except Exception as e:
        st.error(f"Ошибка при сохранении аудио: {str(e)}")
        return None
    
def cleanup_files(*files):
    for file in files:
        try:
            if file and os.path.exists(file):
                os.remove(file)
        except Exception as e:
            st.warning(f"Не удалось удалить файл {file}: {str(e)}")

def extract_embeddings(audio_path, feature_extractor, model, sr=16000):
    """Извлечение эмбеддингов с помощью UniSpeechSat"""
    try:
        waveform, orig_sr = torchaudio.load(audio_path)
        
        if orig_sr != sr:
            resampler = torchaudio.transforms.Resample(orig_sr, sr)
            waveform = resampler(waveform)
        
        inputs = feature_extractor(
            waveform.squeeze().numpy(),
            sampling_rate=sr,
            return_tensors="pt",
            padding=True,
            return_attention_mask=True
        )
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
            embeddings = outputs.hidden_states[-1].mean(dim=1).squeeze().numpy()
        
        return embeddings
    except Exception as e:
        st.error(f"Ошибка при извлечении эмбеддингов: {str(e)}")
        return None

def segment_audio(audio_path, segment_length=3, overlap=1, sr=16000):
    """Сегментация аудио на фрагменты"""
    try:
        y, sr = librosa.load(audio_path, sr=sr)
        duration = len(y) / sr
        segments = []
        
        start = 0.0
        while start + segment_length <= duration:
            end = start + segment_length
            segment = y[int(start*sr):int(end*sr)]
            segments.append((start, end, segment))
            start += segment_length - overlap
        
        return segments
    except Exception as e:
        st.error(f"Ошибка при сегментации аудио: {str(e)}")
        return None

def cluster_speakers(embeddings, n_speakers=None):
    """Кластеризация говорящих"""
    try:
        if n_speakers is None:
            n_speakers = min(5, max(2, len(embeddings) // 3))
        
        clustering = AgglomerativeClustering(
            n_clusters=n_speakers,
            metric='cosine',
            linkage='average'
        ).fit(embeddings)
        
        return clustering.labels_
    except Exception as e:
        st.error(f"Ошибка при кластеризации: {str(e)}")
        return None

def transcribe_with_speakers(whisper_model, audio_path, segments, labels):
    """Транскрипция с определением говорящих"""
    try:
        result = whisper_model.transcribe(audio_path, language="ru")
        tagged_segments = []
        for segment in tqdm(result["segments"]):
            seg_start = segment["start"]
            seg_end = segment["end"]
            speaker = "Unknown"
            for (start, end, _), label in zip(segments, labels):
                if not (seg_end <= start or seg_start >= end):
                    speaker = f"Speaker {label+1}"
                    break
            
            tagged_segments.append({
                "start": seg_start,
                "end": seg_end,
                "text": segment["text"],
                "speaker": speaker
            })
        
        return tagged_segments
    except Exception as e:
        st.error(f"Ошибка при транскрипции: {str(e)}")
        return None

def format_transcription(tagged_segments):
    """Форматирование результата транскрипции"""
    formatted = []
    current_speaker = None
    
    for seg in tqdm(tagged_segments):
        if seg["speaker"] != current_speaker:
            formatted.append(f"\n\n**{seg['speaker']}:**\n")
            current_speaker = seg["speaker"]
        formatted.append(seg["text"] + " ")
    
    return "".join(formatted).strip()

/Users/ivan/develop/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Прогон для записи демо

In [ ]:
audio_path = "output.wav"
whisper_model, feature_extractor, embedding_model = load_models() 
segments = segment_audio(audio_path)

if segments:
    embeddings = []
    for start, end, segment in tqdm(segments):
        segment_path = f"temp_segment_{start:.1f}_{end:.1f}.wav"
        sf.write(segment_path, segment, 48000, 'PCM_24')
        
        emb = extract_embeddings(segment_path, feature_extractor, embedding_model)
        if emb is not None:
            embeddings.append(emb)
        
        cleanup_files(segment_path)
    
    if embeddings:
        print("кластеризуем спикеров")
        labels = cluster_speakers(np.array(embeddings))
        
        if labels is not None:
            print("транскрибируем со спикерами")
            tagged_segments = transcribe_with_speakers(
                whisper_model, audio_path, segments, labels
            ) 
            
            if tagged_segments:
                print("Формирую транскрибацию")
                transcription = format_transcription(tagged_segments)
                print("Вызываю Гигачат")
                analysis = analyze_transcription(transcription)

In [ ]:
print(transcription)

In [ ]:
analysis = analyze_transcription(transcription)

In [ ]:
print(analysis)

Условные роли:
- Speaker 1 — Менеджер проекта / руководитель направления
- Speaker 2 — Специалист аналитического отдела / эксперт по проектам автоматизации
- Speaker 4 — Сотрудник технического подразделения
- Speaker 5 — Участник обсуждения

Основные темы/проблемы:
- Оптимизация процесса обработки заявок на лизинг легковых автомобилей
- Повышение эффективности коммуникации с клиентом путем внедрения автоматизированных инструментов
- Создание быстрого и удобного клиентского пути для оформления лизинга
- Необходимость разработки новой модели взаимодействия с клиентами через сайт Сберлизинга
- Интеграция технологий искусственного интеллекта и чат-ботов для улучшения обслуживания клиентов

Поручения:
- Разработать DS-модель для определения целевых клиентов, готовых взять лизинг легковой техники (готовность к середине июня)
- Подготовить массовую рассылку уведомлениям клиентам о предложениях Сбербанка по лизингу (до конца июня)
- Реализовать механизм расчета стоимости лизинга на сайте Сберл

In [ ]:
with open("transcribe_03_06_full.txt", "w") as file:
    file.write(transcription)

### Пайплайн для полиции

In [ ]:
import os
import subprocess
import numpy as np
from tqdm import tqdm
import soundfile as sf

def download_youtube_audio(url, output_dir="audio"):
    """Скачивает аудио из YouTube видео и сохраняет в указанную директорию"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Извлекаем ID видео из URL
    if "youtube.com/watch" in url:
        video_id = url.split("v=")[1].split("&")[0]
    else:
        video_id = url.split("/")[-1]
    
    output_path = os.path.join(output_dir, f"{video_id}.m4a")
    
    # Команда для youtube-dl
    command = [
        "yt-dlp",
        "-x",  # Извлекаем только аудио
        "--audio-format", "m4a",  # Формат аудио
        "-o", output_path,  # Путь для сохранения
        url
    ]
    
    try:
        subprocess.run(command, check=True)
        return output_path
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при загрузке видео {url}: {e}")
        return None

def process_audio(audio_path):
    """Обрабатывает аудиофайл и возвращает транскрипцию и анализ"""
    whisper_model, feature_extractor, embedding_model = load_models() 
    segments = segment_audio(audio_path)
    
    if not segments:
        return None, None
    
    embeddings = []
    for start, end, segment in tqdm(segments):
        segment_path = f"temp_segment_{start:.1f}_{end:.1f}.wav"
        sf.write(segment_path, segment, 48000, 'PCM_24')
        
        emb = extract_embeddings(segment_path, feature_extractor, embedding_model)
        if emb is not None:
            embeddings.append(emb)
        
        cleanup_files(segment_path)
    
    if not embeddings:
        return None, None
    
    print("Кластеризуем спикеров")
    labels = cluster_speakers(np.array(embeddings))
    
    if labels is None:
        return None, None
    
    print("Транскрибируем со спикерами")
    tagged_segments = transcribe_with_speakers(
        whisper_model, audio_path, segments, labels
    ) 
    
    if not tagged_segments:
        return None, None
    
    print("Формирую транскрибацию")
    transcription = format_transcription(tagged_segments)
    # print("Вызываю Гигачат")
    # analysis = analyze_transcription(transcription)
    analysis = ""
    
    return transcription, analysis

def save_to_markdown(video_url, transcription, analysis, output_dir="transcriptions"):
    """Сохраняет результаты в Markdown файл"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Создаем имя файла из ID видео
    if "youtube.com/watch" in video_url:
        video_id = video_url.split("v=")[1].split("&")[0]
    else:
        video_id = video_url.split("/")[-1]
    
    output_path = os.path.join(output_dir, f"{video_id}.md")
    
    # Формируем Markdown содержимое
    markdown_content = f"# Транскрипция видео: {video_url}\n\n"
    markdown_content += "## Транскрипция:\n\n"
    markdown_content += transcription + "\n\n"
    # markdown_content += "## Анализ:\n\n"
    # markdown_content += analysis + "\n"
    
    # Сохраняем в файл
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)
    
    return output_path

In [ ]:
list_of_videos = ["https://www.youtube.com/watch?v=s6jJxh72MSc",
"https://youtu.be/s_2W-bQ689w",
"https://youtu.be/OiZnJvgUn-M",
'https://youtu.be/jCy9q4OSTww']

for video_url in list_of_videos:
    print(f"\nОбрабатываю видео: {video_url}")
    
    # 1. Скачиваем аудио
    print("Скачиваю аудио...")
    audio_path = download_youtube_audio(video_url)
    if not audio_path:
        print(f"Не удалось скачать аудио для {video_url}")
        continue
    
    # 2. Обрабатываем аудио
    print("Обрабатываю аудио...")
    transcription, analysis = process_audio(audio_path)
    # if not transcription or not analysis:
    #     print(f"Не удалось обработать аудио для {video_url}")
    #     continue
    
    # 3. Сохраняем результаты
    print("Сохраняю результаты...")
    md_path = save_to_markdown(video_url, transcription, analysis)
    print(f"Результаты сохранены в: {md_path}")
    
    # # 4. Удаляем временный аудиофайл (опционально)
    # cleanup_files(audio_path)


Обрабатываю видео: https://www.youtube.com/watch?v=s6jJxh72MSc
Скачиваю аудио...
[youtube] Extracting URL: https://www.youtube.com/watch?v=s6jJxh72MSc
[youtube] s6jJxh72MSc: Downloading webpage
[youtube] s6jJxh72MSc: Downloading tv client config
[youtube] s6jJxh72MSc: Downloading tv player API JSON
[youtube] s6jJxh72MSc: Downloading ios player API JSON
[youtube] s6jJxh72MSc: Downloading m3u8 information
[info] s6jJxh72MSc: Downloading 1 format(s): 251
[download] audio/s6jJxh72MSc.m4a has already been downloaded
[ExtractAudio] Not converting audio audio/s6jJxh72MSc.m4a; file is already in target format m4a
Обрабатываю аудио...


/var/folders/1c/l0_ndlvs3l578q2mzbh87q280000gn/T/ipykernel_9037/1023990989.py:128: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=sr)
/Users/ivan/develop/venv/lib/python3.13/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
100%|██████████| 146/146 [00:04<00:00, 36.21it/s]
/Users/ivan/develop/venv/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Кластеризуем спикеров
Транскрибируем со спикерами


100%|██████████| 130/130 [00:00<00:00, 250924.77it/s]


Формирую транскрибацию


...

Кластеризуем спикеров
Транскрибируем со спикерами


100%|██████████| 126/126 [00:00<00:00, 180000.78it/s]


Формирую транскрибацию


100%|██████████| 126/126 [00:00<00:00, 2565448.08it/s]

Сохраняю результаты...
Результаты сохранены в: transcriptions/jCy9q4OSTww.md


### Подгрузка и проверка модели

In [ ]:
# подгружаем модель large 

import whisper
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def load_whisper_model():
    return whisper.load_model("large-v3")

load_whisper_model()

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm(

In [ ]:
# проверяем модель на песне Газманова 

def transcribe_audio(model, audio_file):
    # Явно указываем язык и задачу
    result = model.transcribe(audio_file, language="russian", task="transcribe")
    return result["text"]

transcribe_audio(model=load_whisper_model(), audio_file="gaz.mp3")

/Users/ivan/develop/venv/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


' Славься, Славься, Белорусская Победа, Россия, Россия, В этом слове огонь и сила, В этом слове победы пламя. Поднимаем России знамя. Россия, Россия, В этом слове огонь и сила, В этом слове победы пламя. Поднимаем России знамя. Пусть время нас бурным потоком несёт, За нами Россия, за нами народ. Традиции святы и тысячи лет, Продолжится летопись наших побед. А если врагов налетит вороньё, Их снова Отечество встретит моё. Россия, Россия, В этом слове огонь и сила, В этом слове победы пламя. Поднимаем России знамя. Россия, Россия, В этом слове огонь и сила, В этом слове победы пламя. Поднимаем России знамя. ... В этом слове источник силы, Повторяем вперед Россия! Россия, Россия, В этом слове огонь и сила, В этом слове победы пламя, Поднимаем Россию знамя! Россия, Россия, В этом слове огонь и сила, В этом слове победы пламя, Поднимаем Россию знамя!'